In [1]:
using Symbolics
using LinearAlgebra

Julia documention: https://docs.julialang.org/en/v1/


# Key generation

In [2]:
@variables x[1:36]
x[36]


# @variables a,b,c,t,x,y
# typeof(a+b)

# B=simplify.([t^2 + t + t^2  2t + 4t;
#                   x + y + y + 2t  x^2 - x^2 + y^2])
# B
x[36]

x[36]

In [6]:
function modify_coeffcients(A,q)
    
    coeffs_map=Symbolics.value(A).dict#turn the polynomial into matrix
    B=0#create our new polynomial
    for (key, value) in coeffs_map#iterate through the dict(monomial, coeffcients)
        if value>=q
            new_coef=value%q
            B+=new_coef*key
        else
            B+=key*value
        end  
    end
    B
end

modify_coeffcients (generic function with 1 method)

In [60]:
#t monomials
#bound the degree of mononials by b
#coefficients go from 1 to q-1(inclusive)
function sparse_polynomial(n,t,b,q)
    @variables x[1:n]
    
    sparse_poly=0
    
    for i in 1:t #we want t monomials
        deg=rand(1:b)
        
        monomial_d=1
        for j in 1:deg
            monomial_d*=x[rand(1:n)]
        end
        sparse_poly+=rand(1:(q-1))*monomial_d
        
    end
#     println(sparse_poly)
    A=modify_coeffcients(sparse_poly,q)
    return A
end

A=sparse_polynomial(4,5,3,6)
A

3x[1] + 5x[2]*x[3] + 5x[2]*x[4] + 3x[2]*x[3]*x[4]

# matrix utility functions

In [41]:
A1 = [1 2 3; 4 1 6; 7 8 1]
det(A1)
I=diagm(0=>ones(3))#scalar*Identity matrix
I


3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [61]:
function num_identity(k)
    a=zeros(Num,k)
    a[1]=1
    for i in 2:k
        b=zeros(Num,k)
        b[i]=1
        a=[a b] 
    end
    return a
end
I_mat=num_identity(4)
I_mat[1,1]=x[2]*x[3]+x[1]
I_mat



4×4 Matrix{Num}:
 x[2]*x[3] + x[1]  0  0  0
                0  1  0  0
                0  0  1  0
                0  0  0  1

In [67]:
function generate_U(k,n,t,B,q)
    I_matrix=num_identity(k)#generate a k by k identity matrix
    for i in 1:Int((k*k-k)/2)
        b=rand(2:k)
        a=rand(1:(b-1))
        I_matrix[a,b]=sparse_polynomial(n,t,B,q)
    end

    return I_matrix
end 

U=generate_U(4,32,4,3,6)
U
# println(U)
simplify.(expand.(inv(U)*U))


4×4 Matrix{Num}:
 1.0  0    0    0
 0.0  1.0  0    0
 0.0  0.0  1.0  0
 0.0  0.0  0.0  1.0

In [68]:
function generate_L(k,n,t,B,q)
    I_matrix=num_identity(k)#generate a k by k identity matrix
    for i in 1:Int((k*k-k)/2)
        a=rand(2:k)
        b=rand(1:a-1)
        I_matrix[a,b]=sparse_polynomial(n,t,B,q)
    end
    return I_matrix
end
L=generate_L(4,32,4,3,6)
expand.(simplify.(inv(L)*L))

# substitute.(inv(U)*U,(substitution_dict,))

4×4 Matrix{Num}:
 1.0  0.0  0.0  0.0
 0    1.0  0.0  0.0
 0    0.0  1.0  0.0
 0    0    0.0  1.0

In [213]:
A=[]
push!(A,1)
push!(A,'a')
println(A)
B=["kk",1]
vcat(A,B)
[i for i in 1:3]
zeros(Num,3,4)[:,2]

Any[1, 'a']


3-element Vector{Num}:
 0
 0
 0

# utility functions

In [73]:
function random_del_cols(k,l,matrix)
    col_opts=[i for i in 1:l]
    matrix_kl=zeros(Num, k,l)
    
    j=1
    for i in col_opts
        matrix_kl[:,j]=matrix[:,i]
        j+=1
    end
    return col_opts,matrix_kl
end

random_del_cols(4,2,[5 6 6 8; 2 2 2 8; 6 6 2 8; 2 3 6 7])
# size([5 6 6 8; 2 2 2 8; 6 6 2 8; 2 3 6 7],2)
size([5 6; 2 2; 6 6; 2 3])

(4, 2)

In [75]:
function find_kl_inverse(to_keep,S_inv)
    k=size(S_inv,1)#find the number of rows
    ml_inv=zeros(Num, length(to_keep),k)
    
    j=1
    for i in to_keep
        ml_inv[j,:]=S_inv[i,:]
        j+=1
    end
    return ml_inv
end
find_kl_inverse([1 2],[-17 -9 12 16;17 35/4 -47/4 -16;-4 -9/4 11/4 4; 1 3/4 -3/4 -1])*[5 6; 2 2; 6 6; 2 3]
    

2×2 Matrix{Num}:
 1.0  0.0
 0.0  1.0

In [261]:
1==1 && 4+5==9

true

In [103]:
function modify_neg_coeff(A,q)
    
    coeffs_map=Symbolics.value(A).dict#turn the polynomial into matrix
    B=0#create our new polynomial
    for (key, value) in coeffs_map#iterate through the dict(monomial, coeffcients)
        if value>=q || value<0 
            new_coef=value%6
            B+=new_coef*key
        else
            B+=key*value
        end  
    end
    B
end


modify_neg_coeff(-4(x[10]^2)*x[5]*x[25]*x[26] - 4x[10]*x[13]*x[25]*x[29] - 8x[4]*x[12]*x[21]*x[25]*x[28] - 5(x[1]^3)*(x[10]^2)*x[5]*x[8]*x[20]*x[26] - 5(x[1]^3)*x[8]*x[10]*x[13]*x[20]*x[29] - 10(x[1]^3)*x[4]*x[8]*x[12]*x[20]*x[21]*x[28] - (x[13]^2)*x[5]*x[10]*x[22]*x[23]*x[29]*x[30] - (x[5]^2)*(x[10]^2)*x[13]*x[22]*x[23]*x[26]*x[30] - 2x[4]*x[5]*x[12]*x[13]*x[21]*x[22]*x[23]*x[28]*x[30],6)

# modify_neg_coeff((x[10]^2)*x[5]*x[26] + x[10]*x[13]*x[29] + 2.0x[4]*x[12]*x[21]*x[28],6)


-4(x[10]^2)*x[5]*x[25]*x[26] - 4x[10]*x[13]*x[25]*x[29] - 2x[4]*x[12]*x[21]*x[25]*x[28] - 5(x[1]^3)*(x[10]^2)*x[5]*x[8]*x[20]*x[26] - 5(x[1]^3)*x[8]*x[10]*x[13]*x[20]*x[29] - 4(x[1]^3)*x[4]*x[8]*x[12]*x[20]*x[21]*x[28] - (x[13]^2)*x[5]*x[10]*x[22]*x[23]*x[29]*x[30] - (x[5]^2)*(x[10]^2)*x[13]*x[22]*x[23]*x[26]*x[30] - 2x[4]*x[5]*x[12]*x[13]*x[21]*x[22]*x[23]*x[28]*x[30]

In [306]:
Num(0)!=0.0
isone((x[10]^2)*x[5]*x[26] + x[10]*x[13]*x[29] + 2.0x[4]*x[12]*x[21]*x[28])
iszero((x[10]^2)*x[5]*x[26] + x[10]*x[13]*x[29] + 2.0x[4]*x[12]*x[21]*x[28])
isone(1.0)==true

true

In [104]:
function modify_matrix_coef(matrix,q)
    res=simplify.(expand.(matrix))#expand and then simplify
    
    for i in 1:size(res,1)
        for j in 1:size(res,2)
            a=res[i,j]

            if isone(a)==false && iszero(a)==false            
                res[i,j]=modify_neg_coeff(a,q)
            end
        end
    end
    return res
end

modify_matrix_coef([1.0 0 0 0.0; 0 1.0 0 0.0; 0 0 1 0.0; -4(x[10]^2)*x[5]*x[25]*x[26] - 4x[10]*x[13]*x[25]*x[29] - 8x[4]*x[12]*x[21]*x[25]*x[28] - 5(x[1]^3)*(x[10]^2)*x[5]*x[8]*x[20]*x[26] - 5(x[1]^3)*x[8]*x[10]*x[13]*x[20]*x[29] - 10(x[1]^3)*x[4]*x[8]*x[12]*x[20]*x[21]*x[28] - (x[13]^2)*x[5]*x[10]*x[22]*x[23]*x[29]*x[30] - (x[5]^2)*(x[10]^2)*x[13]*x[22]*x[23]*x[26]*x[30] - 2x[4]*x[5]*x[12]*x[13]*x[21]*x[22]*x[23]*x[28]*x[30] (x[10]^2)*x[5]*x[26] + x[10]*x[13]*x[29] + 2.0x[4]*x[12]*x[21]*x[28] 0 0.0],6)

4×4 Matrix{Num}:
                                                                                                                                                                                                                                                                                                                                                                                   1.0  …  0  0.0
                                                                                                                                                                                                                                                                                                                                                                                   0       0  0.0
                                                                                                                                                                                                                   

In [125]:
function matrix_kl(k,l,s,n,t,B,q)
    Ls=[]
    Us=[]
    for i in 1:s
        U=generate_U(k,n,t,B,q)
        L=generate_L(k,n,t,B,q)
        push!(Us,U)
        push!(Ls,L)
    end 
    U_new=Us[rand(1:s)]
    L_new=Ls[rand(1:s)]
    kk_matrix=L_new*U_new
    #kk_inv=inv(kk_matrix)#able to compute, not able to verify the result
    kk_inv=inv(U_new)*inv(L_new)
    
    #make modifications to the coefficients of each entry in kk_matrix and kk_inv
#     kk_matrix=modify_matrix_coef(kk_matrix,q)
#     kk_inv=modify_matrix_coef(kk_inv,q)
    
    col_keep,kl_matrix=random_del_cols(k,l,kk_matrix)
    kl_inverse=find_kl_inverse(col_keep,kk_inv)
    
    #modify coefficients
#     kl_matrix=modify_matrix_coef(kl_matrix,q)
#     kl_inverse=modify_matrix_coef(kl_inverse,q)
    
    return kk_matrix,kk_inv,kl_matrix, kl_inverse
end 


A,B,C,D=matrix_kl(4,3,5,32,3,5,6)

println(size(C))
print(size(D))


(4, 3)
(3, 4)

In [239]:
@variables a,y,z
simplify(expand((a+y+z)^3))

a^3 + y^3 + z^3 + 3z*(a^2 + y^2) + 3a*(y^2 + z^2) + 3y*(a^2 + z^2) + 6a*y*z

In [126]:
res=B*A#please do inverse of kl matrix * kl matrix


4×4 Matrix{Num}:
 (2x[23] + x[2]*x[22]*x[31] + 3x[11]*x[19]*x[31])*(2(x[8]^3)*x[4]*x[11]*x[20]*x[25] + 2(x[8]^4)*x[4]*x[11]*x[15]*x[20] + 4(x[8]^4)*x[4]*x[11]*x[20]*x[32] + 2(x[8]^3)*(x[24]^2)*x[4]*x[11]*x[15]*x[17] + 2(x[8]^2)*(x[24]^2)*x[4]*x[11]*x[17]*x[25] + 4(x[8]^3)*(x[24]^2)*x[4]*x[11]*x[17]*x[32] + 2(x[8]^2)*(x[17]^2)*(x[24]^3)*x[1]*x[4]*x[11]*x[15] + 4(x[8]^2)*(x[17]^2)*(x[24]^3)*x[1]*x[4]*x[11]*x[32] + 2(x[17]^2)*(x[24]^3)*x[1]*x[4]*x[8]*x[11]*x[25] + 4(x[8]^2)*(x[25]^2)*x[3]*x[4]*x[9]*x[26]*x[31] + 4(x[8]^3)*(x[11]^2)*x[4]*x[5]*x[20]*x[21]*x[24]*x[32] + 4(x[8]^3)*x[1]*x[4]*x[11]*x[17]*x[20]*x[24]*x[32] + 2(x[11]^2)*(x[24]^3)*x[4]*x[5]*x[8]*x[17]*x[21]*x[25] + 4(x[8]^3)*x[3]*x[4]*x[9]*x[15]*x[25]*x[26]*x[31] + 2(x[8]^3)*x[1]*x[4]*x[11]*x[15]*x[17]*x[20]*x[24] + 2(x[8]^2)*(x[11]^2)*(x[24]^3)*x[4]*x[5]*x[15]*x[17]*x[21] + 2(x[8]^2)*(x[11]^2)*x[4]*x[5]*x[20]*x[21]*x[24]*x[25] + 2(x[8]^3)*x[3]*x[4]*x[9]*x[25]*x[26]*x[31]*x[32] + 4(x[8]^2)*(x[11]^2)*(x[24]^3)*x[4]*x[5]*x[17]*x[21]

In [127]:
res=simplify(expand.(res))


4×4 Matrix{Num}:
                                                                                                                                                                    6.0(x[8]^4)*x[4]*x[11]*x[20]*x[23]*x[32] + 6.0(x[8]^3)*(x[24]^2)*x[4]*x[11]*x[17]*x[23]*x[32] + 6.0(x[8]^4)*(x[11]^2)*x[4]*x[15]*x[19]*x[20]*x[31] + 6.0(x[8]^3)*(x[11]^2)*x[4]*x[19]*x[20]*x[25]*x[31] + 12.0(x[8]^4)*(x[11]^2)*x[4]*x[19]*x[20]*x[31]*x[32] + 6.0(x[8]^2)*(x[11]^2)*(x[24]^2)*x[4]*x[17]*x[19]*x[25]*x[31] + 6.0(x[8]^2)*(x[25]^2)*x[3]*x[4]*x[9]*x[23]*x[26]*x[31] + 6.0(x[8]^3)*(x[11]^2)*(x[24]^2)*x[4]*x[15]*x[17]*x[19]*x[31] + 6.0(x[8]^2)*(x[17]^2)*(x[24]^3)*x[1]*x[4]*x[11]*x[23]*x[32] + 12.0(x[8]^3)*(x[11]^2)*(x[24]^2)*x[4]*x[17]*x[19]*x[31]*x[32] + 6.0(x[8]^2)*(x[11]^2)*(x[17]^2)*(x[24]^3)*x[1]*x[4]*x[15]*x[19]*x[31] + 6.0(x[8]^2)*(x[11]^2)*(x[24]^3)*x[4]*x[5]*x[17]*x[21]*x[23]*x[32] + 6.0(x[11]^2)*(x[17]^2)*(x[24]^3)*x[1]*x[4]*x[8]*x[19]*x[25]*x[31] + 6.0(x[8]^3)*(x[11]^2)*x[4]*x[5]*x[20]*x[21]*x[

# Substitution to verify

In [128]:
substitution_dict=Dict()
for i in 1:36
    substitution_dict[x[i]]=rand(1:10)
end 
substitution_dict

Dict{Any, Any} with 36 entries:
  x[29] => 6
  x[8]  => 4
  x[31] => 7
  x[36] => 9
  x[6]  => 7
  x[17] => 10
  x[28] => 6
  x[13] => 6
  x[33] => 9
  x[11] => 8
  x[16] => 10
  x[20] => 3
  x[26] => 2
  x[34] => 10
  x[30] => 8
  x[7]  => 8
  x[27] => 1
  x[21] => 2
  x[5]  => 7
  x[1]  => 1
  x[9]  => 6
  x[12] => 7
  x[3]  => 8
  x[4]  => 1
  x[18] => 5
  ⋮     => ⋮

In [129]:

substitute.(res,(substitution_dict,))

4×4 Matrix{Num}:
       1.55673e12          6.48805e14  …  -18710894077248
      -6.82867e9          -2.84526e12                   8.21571e10
       2.06992e8           8.56356e10                   1.39438e7
 -167250           -51894099                     -2728572